In [ ]:
import os, string, glob, random, shutil, base64
import ggl_img_scraper as ggl
from PIL import Image
from tqdm import tqdm
from utils import *

In [ ]:
# Necessary inputs
ggl_api_key = ''
search_engine_id = 'f1aca5d66c8d4435c'

raw_dir = '..\\data\\raw\\'
training_dir = '..\\data\\training\\'
validation_dir = '..\\data\\validation\\'

validation_split = .15  # Takes images for validation

num_images = 5
birds_txt = '..\\data\\bird_lists\\bay_area_birds.txt'
db_name = ".\\bird_im_urls.db"

clear_dirs = True
remove_db = True

In [ ]:
## Clear dir
if clear_dirs:
    clear_dir(raw_dir, [])
    clear_dir(training_dir, [])
    clear_dir(validation_dir, [])
if remove_db and os.path.exists(db_name):
    os.remove(db_name)
    if os.path.exists(db_name+"-journal"):
        os.remove(db_name+"-journal")

In [ ]:
# Read-in txt file of bird names
# File must be new-line delimited
birds = []
with open(birds_txt) as f:
    if f.readable() is False:
        raise FileNotFoundError("ERROR: File is not a readable file.")
    birds = f.readlines()
birds = [ string.capwords(x.strip()) for x in birds]
birds.sort()

In [ ]:
# Make a directory for every bird in the list in the training and validation directories
for bird in birds:
    if not os.path.exists(raw_dir + bird + '\\'):
        os.makedirs(raw_dir + bird + '\\')
    if not os.path.exists(training_dir + bird + '\\'):
        os.makedirs(training_dir + bird + '\\')
    if not os.path.exists(validation_dir + bird + '\\'):
        os.makedirs(validation_dir + bird + '\\')

In [ ]:
# Get Images for each bird from Google Images
for bird in tqdm(birds):
    search_query = '"' + bird + '"' + " bird"
    save_dir = raw_dir + '\\'+bird+'\\'
    saved = ggl.google_image_download(query=search_query, save_directory=save_dir, api_key=ggl_api_key, cx=search_engine_id, n=num_images, name=bird, db_name=db_name,delay=100)
    if len(saved) != num_images:
        print(saved)
        print(bird + " saved " + str(len(saved)) + " not " + str(num_images))

In [ ]:
# Counts directories for correct number of images
count_jpg_images(raw_dir, num_images, raise_e = False)

In [ ]:
# Normalize Every Image to RGB
for image_path in glob.glob(raw_dir + '*\\*.jpg'):
    try:
        image = Image.open(image_path)
        image = image.convert("RGB")
        image.save(image_path.replace(raw_dir, training_dir))
    except Exception as e:
        print(f"ERROR: ", image_path)
        print(e)

In [ ]:
# Make Training and Validation Split
for bird in birds:
    val_imgs = []
    num = None
    while len(val_imgs) < round(num_images * validation_split):
        num = random.randint(1, num_images)
        if num not in val_imgs:
            val_imgs.append(num)
            shutil.move(os.path.join(training_dir,bird, str(num)+"__"+bird+ '.jpg'),
                        os.path.join(validation_dir,bird, str(num)+"__"+bird+ '.jpg'))
        